In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(2,5)
        self.l2 = nn.Linear(5,5)
        self.l3 = nn.Linear(5,1)
        
    def forward(self, x):
        x = self.l1(x)
        x = F.sigmoid(x)
        x = self.l2(x)
        x = F.sigmoid(x)
        x = self.l3(x)
        x = F.sigmoid(x)
        return x

In [ ]:
DATA_SIZE = 100000
BATCH_SIZE = 10
data_in = np.random.random((DATA_SIZE, 2))
data_in[data_in > 0.5] = 1
data_in[data_in < 0.5] = 0
data_in = np.array(data_in, np.float32)
data_out = np.expand_dims(np.logical_and(data_in[:,0], data_in[:,1])*1.0, -1)
data_out = np.array(data_out, np.float32)

model = Model()

plot_steps = []
plot_losses = []

loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

for step in tqdm(range(DATA_SIZE//BATCH_SIZE)):

    x = torch.tensor(data_in[(step+0)*BATCH_SIZE:(step+1)*BATCH_SIZE])
    t = torch.tensor(data_out[(step+0)*BATCH_SIZE:(step+1)*BATCH_SIZE])

    y = model(x)

    loss = torch.sum((t-y)**2)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    plot_steps.append(step)
    plot_losses.append(loss.detach().numpy())

In [ ]:
plt.scatter(plot_steps, plot_losses, alpha=0.01)